In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl as px
from sklearn.model_selection import train_test_split

AAPL = pd.read_excel("./data/AAPL_EQUITY.xlsx")
SPESG = pd.read_excel(('./data/SPESG_Index_5Y.xlsx')).dropna()

# Take SPESG and AAPL and combine them into one data frame: df['Date', 'SPESG', 'AAPL_Px', 'AAPL_Volume', 'AAPL_SMAVG(15)']
df = pd.merge(AAPL, SPESG, on='Date', how='inner')
new_columns = {'Last Px_x' : 'AAPL_Px', 'Volume': 'AAPL_Volume(M)', 'SMAVG(15)' : 'AAPL_SMAVG15(M)', 'Last Px_y': 'SPESG'}
df.rename(columns=new_columns, inplace=True)

# Strip M and convert these columns from object into float64
df['AAPL_Volume(M)'] = df['AAPL_Volume(M)'].str.strip('M').astype('float64').shift(-1)
df['AAPL_SMAVG15(M)'] = df['AAPL_SMAVG15(M)'].str.strip('M').astype('float64').shift(-1)
df['SPESG-1'] = df['SPESG'].shift(-1)
df['AAPL_Px_Shift-1'] = df['AAPL_Px'].shift(-1)
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Quarter'] = df['Date'].dt.quarter
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.strftime('%A')
weekday_dummies = pd.get_dummies(df['Day']).astype(int)
df = pd.concat([df, weekday_dummies], axis=1)
df = df.dropna()
df.sort_values(by='Date', ascending=True, inplace=True)

# features = ['AAPL_Volume(M)', 'AAPL_SMAVG15(M)', 'SPESG', 'Month', 'Year', 'Day']
X = df.drop(['Date', 'AAPL_Px', 'Day', 'SPESG'], axis=1)
y = df['AAPL_Px']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)
X_test.shape

(1134, 12)

In [53]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regressor = linear_model.LinearRegression()
regressor.fit(X_train, y_train)
# Training set
y_pred_train = regressor.predict(X_train)
r2_train = r2_score(y_train, y_pred_train)

# Test set
y_pred_test = regressor.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)

print(f"r2_train: {r2_train}")
print(f"r2_test: {r2_test}")

def adjusted_r2_score(r_squared, n, k):
    return 1 - ((1 - r_squared) * (n - 1) / (n - k - 1))

n_samples = X_train.shape[0]
n_features = X_train.shape[1]

adj_r2_train = adjusted_r2_score(r2_train, n_samples, n_features)
adj_r2 = adjusted_r2_score(r2_test, n_samples, n_features)
print("Adjusted R-squared on Training Data:", adj_r2_train)
print("Adjusted R-squared on Test Data:", adj_r2)

r2_train: 0.9976413449500031
r2_test: 0.996617719339054
Adjusted R-squared on Training Data: 0.997388631908932
Adjusted R-squared on Test Data: 0.9962553321253812


In [54]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Assuming y_true_train, y_pred_train, y_true_test, y_pred_test are your actual and predicted values
# for both the training and test sets

# Mean Absolute Error (MAE)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

# Mean Squared Error (MSE)
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

# Root Mean Squared Error (RMSE)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

# R2 Score
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print("MAE - Train: {:.4f}, Test: {:.4f}".format(mae_train, mae_test))
print("MSE - Train: {:.4f}, Test: {:.4f}".format(mse_train, mse_test))
print("RMSE - Train: {:.4f}, Test: {:.4f}".format(rmse_train, rmse_test))
print("R2 Score - Train: {:.4f}, Test: {:.4f}".format(r2_train, r2_test))

MAE - Train: 1.5202, Test: 1.8394
MSE - Train: 4.3581, Test: 6.4305
RMSE - Train: 2.0876, Test: 2.5358
R2 Score - Train: 0.9976, Test: 0.9966


In [55]:
X_test.head()

,AAPL_Volume(M),AAPL_SMAVG15(M),SPESG-1,AAPL_Px_Shift-1,Year,Quarter,Month,Friday,Monday,Thursday,Tuesday,Wednesday
1182,146.190,141.621,237.25,50.688,2019,3,8,0,0,1,0,0
232,50.747,57.523,367.36,171.560,2023,2,5,0,0,0,0,1
1215,83.599,96.118,243.01,49.935,2019,2,6,1,0,0,0,0
592,131.063,120.621,410.26,179.300,2021,4,12,0,0,1,0,0
729,59.279,78.056,358.08,125.060,2021,2,6,0,0,1,0,0
